In [1]:
import pandas as pd
import numpy as np
import imblearn as imb
pd.set_option('display.max_columns', None)

In [47]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

C:\Continuum\Anaconda2\envs\TFENV\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,307,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Continuum\Anaconda2\envs\TFENV\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
train.shape

(300000, 377)

In [49]:
#train.head(5)

In [50]:
train.Responders.value_counts()

0    248303
1     51697
Name: Responders, dtype: int64

In [51]:
train.Responders.isnull().sum()

0

In [52]:
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns

In [53]:
mis_val_table_ren_columns = missing_values_table(train)

In [54]:
nan_col = list(mis_val_table_ren_columns[mis_val_table_ren_columns['% of Total Values']> 90].index)

In [55]:
print(len(nan_col))

91


In [56]:
y_train = train['Responders']
train.drop(['Responders'],axis = 1,inplace=True)

In [57]:
train.drop(nan_col, inplace=True, axis=1)

In [58]:
test.drop(nan_col, inplace=True, axis=1)

In [59]:
cor = train.corr()

In [60]:
i=0
cols = []
for col in cor.columns.values:
    for index in cor[col].index:
        if(cor.get_value(index,col) > 0.80 or cor.get_value(index,col) < -0.80) and (index != col ) and sorted((index,col)) not in cols:
            i+=1
            tpl = (index,col)
            cols.append(sorted(tpl))
            #print ( i,sorted(tpl),'=> ',cor.get_value(index,col))

In [61]:
cols_0= [i[0] for i in cols]
cols_1= [i[1] for i in cols]
#cols_0 = list(set(cols_0))
cols_0.sort()
#cols_1 = list(set(cols_1))
cols_1.sort()
col_rem=[]
for col in cols_0:
    if col in cols_1:
        col_rem.append(col)
col_rem = list(set(col_rem))

cols_0 = list(set(cols_0))
for col in col_rem:
    if col in cols_0:
        cols_0.remove(col)
len(col_rem)

93

In [62]:
train.drop(col_rem, axis=1,inplace=True)
test.drop(col_rem,axis=1,inplace=True)

In [63]:
len(cols_0)

31

In [64]:
train.drop(cols_0, axis=1,inplace=True)
test.drop(cols_0,axis=1,inplace=True)

In [65]:
train.shape

(300000, 161)

In [66]:
obj_cols = list(train.select_dtypes(include = ['O']).columns)

In [67]:
obj_cols

['HNW_CATEGORY',
 'OCCUP_ALL_NEW',
 'city',
 'FINAL_WORTH_prev1',
 'ENGAGEMENT_TAG_prev1',
 'EFT_SELF_TRANSFER_PrevQ1',
 'CC_TAG_LIVE',
 'DEMAT_TAG_LIVE',
 'SEC_ACC_TAG_LIVE',
 'INS_TAG_LIVE',
 'FD_TAG_LIVE',
 'Billpay_Active_PrevQ1_N',
 'Billpay_Reg_ason_Prev1_N',
 'Charges_cnt_PrevQ1_N',
 'FRX_PrevQ1_N',
 'RBI_Class_Audit',
 'gender_bin']

In [39]:
for col in obj_cols:
    #print(col)
    train[col] = train[col].astype('category').cat.codes

In [84]:
def confusion_mat(train,y_train,col,val): 
    m_y=[]
    m_n=[]
    for i in range(len(val)):
        m_y.append(0)
        m_n.append(0)
    
    for i in train.index.values:
        for j in range(len(val)):
            if( train.get_value(i,col) == val[j]) and (y_train.get_value(i) == 1):
                m_y[j]+=1
            elif( train.get_value(i,col) == val[j]) and (y_train.get_value(i) == 0):
                m_n[j]+=1
    print ('\tyes','\tno','\t%yes','\t%no','\n')
    for i in range(len(val)):
        print (val[i],'\t',m_y[i],'\t',m_n[i],'\t',100 * m_y[i]/sum(m_y),'%','\t',100 * m_n[i]/sum(m_n),'%','\n')

In [76]:
for col in obj_cols:
    if col != 'city':
        print(col , list(train[col].unique()))

HNW_CATEGORY ['2_Preferred', '1_Imperia', '3_Classic']
OCCUP_ALL_NEW ['HOUSEWIFE', 'SALARIED', 'SELF_EMPLOYED', 'INDIVIDUAL', 'RETIRED', 'STUDENT', 'NON_INDIVIDUA', 'MISSING', nan]
FINAL_WORTH_prev1 ['HIGH', 'MEDIUM', 'LOW', nan]
ENGAGEMENT_TAG_prev1 ['LOW', 'MEDIUM', 'HIGH', nan, 'NO']
EFT_SELF_TRANSFER_PrevQ1 ['N', 'Y']
CC_TAG_LIVE [nan, 'Y']
DEMAT_TAG_LIVE [nan, 'Y']
SEC_ACC_TAG_LIVE [nan, 'Y']
INS_TAG_LIVE [nan, 'Y']
FD_TAG_LIVE [nan, 'Y']
Billpay_Active_PrevQ1_N ['A_MISSING', 'B_1']
Billpay_Reg_ason_Prev1_N ['A_MISSING', 'B_1']
Charges_cnt_PrevQ1_N ['A_MISSING', 'D_3', 'C_2', 'B_1']
FRX_PrevQ1_N ['A_MISSING', 'B_1']
RBI_Class_Audit ['SEMI-URBAN', 'METROPOLITAN', 'URBAN', 'RURAL']
gender_bin ['Female', 'Male', 'Missin']


In [87]:
confusion_mat(train,y_train,'gender_bin',list(train['gender_bin'].unique()))

	yes 	no 	%yes 	%no 

Female 	 19624 	 99588 	 37.95964949610229 % 	 40.10744936629843 % 

Male 	 30908 	 143919 	 59.786834826005375 % 	 57.9610395363729 % 

Missin 	 1165 	 4796 	 2.253515677892334 % 	 1.9315110973286669 % 



In [86]:
confusion_mat(train,y_train,'HNW_CATEGORY',list(train['HNW_CATEGORY'].unique()))

	yes 	no 	%yes 	%no 

2_Preferred 	 26073 	 128476 	 50.43426117569685 % 	 51.7416221310254 % 

1_Imperia 	 6545 	 31828 	 12.660309108845775 % 	 12.81821000954479 % 

3_Classic 	 19079 	 87999 	 36.90542971545737 % 	 35.440167859429806 % 



In [85]:
confusion_mat(train,y_train,'OCCUP_ALL_NEW',list(train['OCCUP_ALL_NEW'].unique()))

	yes 	no 	%yes 	%no 

HOUSEWIFE 	 9745 	 47764 	 18.856059286778507 % 	 19.242374800078963 % 

SALARIED 	 11290 	 59279 	 21.845552524138466 % 	 23.881348626033848 % 

SELF_EMPLOYED 	 20240 	 79767 	 39.16332888295505 % 	 32.13521712331251 % 

INDIVIDUAL 	 4101 	 23659 	 7.935217971788472 % 	 9.531348827465626 % 

RETIRED 	 2883 	 18143 	 5.578452429326058 % 	 7.30915346281368 % 

STUDENT 	 3238 	 18603 	 6.26535864244113 % 	 7.49447069771939 % 

NON_INDIVIDUA 	 60 	 387 	 0.11609682475184303 % 	 0.15590819545328194 % 

MISSING 	 124 	 621 	 0.2399334378204756 % 	 0.2501782671227082 % 

nan 	 0 	 0 	 0.0 % 	 0.0 % 



In [88]:
for col in train.columns.values:
    train[col] = train[col].apply(lambda x: x if str(x) != str(pd.np.nan) else -1)
    test[col] = test[col].apply(lambda x: x if str(x) != str(pd.np.nan) else -1)